# Sentiment Analysis with Transformer Fine-Tuning

This project builds a sentiment classification system for movie reviews.

Originally, the project began with a classical machine learning approach
(TF-IDF + Logistic Regression).

However, to build a more rigorous and modern system capable of understanding
grammar and contextual meaning, the model was upgraded to a Transformer-based
architecture (DistilBERT).

Final model: Fine-tuned DistilBERT for binary sentiment classification.


## 1. Dataset

The dataset consists of labeled movie reviews:

- `Review`: raw text input
- `Emotion`: sentiment label (positive / negative)

We perform supervised learning using these labeled examples.


In [ ]:
!wget -O movie_reviews 'https://drive.google.com/uc?export=view&id=1kWs6yOYpdjVr-liLPs4PKIs1qSpIohzS'

In [ ]:
## Label Data

import numpy as np
import pandas as pd
data = pd.read_csv('movie_reviews', delimiter=",")
data.head(20)


# 2. Dataset Inspection

We inspect:

- Total dataset size
- Class distribution
- Label balance

This ensures the dataset is suitable for supervised learning.


In [ ]:
x = data['Review']
y = data['Emotion']

In [ ]:
!pip -q install transformers datasets accelerate torch scikit-learn


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

# 3. Label Encoding

Sentiment labels are mapped to numeric IDs to allow
training using cross-entropy loss.


In [ ]:
data = data[["Review", "Emotion"]].dropna()

# Map labels to ids
labels = sorted(data["Emotion"].unique())
label2id = {l: i for i, l in enumerate(labels)}
id2label = {i: l for l, i in label2id.items()}

data["label"] = data["Emotion"].map(label2id)

print("Labels:", labels)
print(data["Emotion"].value_counts())
print("Total rows:", len(data))



# 4. Train-Test Split

The dataset is split into:

- 80% training data
- 20% testing data

Stratified sampling preserves class proportions.


In [ ]:
train_df, test_df = train_test_split(
    data,
    test_size=0.2,
    random_state=20,
    stratify=data["label"]
)

print("Train:", len(train_df), "Test:", len(test_df))



In [ ]:
train_ds = Dataset.from_pandas(train_df[["Review", "label"]])
test_ds  = Dataset.from_pandas(test_df[["Review", "label"]])

# 5. Tokenization

Raw text is converted into token IDs using a pretrained
DistilBERT tokenizer.

This enables contextual embedding and grammar-aware modeling.


In [ ]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(
        batch["Review"],
        truncation=True,
        padding="max_length",
        max_length=256
    )

train_ds = train_ds.map(tokenize, batched=True)
test_ds  = test_ds.map(tokenize, batched=True)

cols = ["input_ids", "attention_mask", "label"]
train_ds.set_format(type="torch", columns=cols)
test_ds.set_format(type="torch", columns=cols)


# 6. Model Architecture

We fine-tune DistilBERT, a Transformer-based neural network.

Architecture:

- Pretrained DistilBERT encoder
- Linear classification head
- Softmax output layer

Training objective: minimize cross-entropy loss.


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
)

def compute_metrics(eval_pred):
    logits, y_true = eval_pred
    y_pred = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "f1_macro": f1_score(y_true, y_pred, average="macro")
    }


In [ ]:
import transformers, inspect
from transformers import TrainingArguments

print("transformers version:", transformers.__version__)
print("transformers file:", transformers.__file__)
print("TrainingArguments init params contains evaluation_strategy?",
      "evaluation_strategy" in str(inspect.signature(TrainingArguments.__init__)))
print("TrainingArguments init params contains eval_strategy?",
      "eval_strategy" in str(inspect.signature(TrainingArguments.__init__)))


# 7. Training Configuration

Hyperparameters:

- Learning rate: 2e-5
- Batch size: 16
- Epochs: 2
- Optimizer: AdamW
- Metric: F1 Macro

GPU acceleration is used for efficient fine-tuning.


In [ ]:
args = TrainingArguments(
    output_dir="sentiment_bert",
    eval_strategy="epoch",     # <-- FIXED
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    report_to="none"
)


In [ ]:
from transformers import DataCollatorWithPadding, Trainer

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# 8. Model Training & Evaluation

The pretrained transformer is fine-tuned
on the labeled movie review dataset.

Final performance metrics:

- Accuracy ≈ 91%
- F1 Macro ≈ 91%

The model demonstrates strong generalization performance.


In [ ]:
trainer.train()
trainer.evaluate()


 # 9. Example of Prediction

In [ ]:
import torch
import torch.nn.functional as F

def predict_with_proba(texts):
    if isinstance(texts, str):
        texts = [texts]

    inputs = tokenizer(texts, return_tensors="pt", truncation=True, padding=True, max_length=256)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    model.eval()
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = F.softmax(logits, dim=-1).cpu().numpy()

    preds = probs.argmax(axis=1)

    for i, text in enumerate(texts):
        pred_label = id2label[int(preds[i])]
        prob_dict = {id2label[j]: float(probs[i][j]) for j in range(len(labels))}
        print("\nTEXT:", text)
        print("PRED:", pred_label)
        print("PROBS:", prob_dict)

predict_with_proba([
    "This movie is not as good.",
    "Absolutely fantastic — I loved it."
])


# 10. Interactive Prediction Demo

Users can input custom movie reviews
to observe real-time sentiment predictions
and confidence scores.


In [ ]:
import torch
import torch.nn.functional as F

def predict_texts(texts):
    if isinstance(texts, str):
        texts = [texts]

    inputs = tokenizer(
        texts,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=256
    )

    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    model.eval()
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = F.softmax(logits, dim=-1).cpu().numpy()

    for text, p in zip(texts, probs):
        pred_id = int(p.argmax())
        pred_label = id2label[pred_id]

        print("\n" + "="*60)
        print("TEXT:", text)
        print("PREDICTION:", pred_label)
        print("CONFIDENCE:", round(float(p[pred_id]), 4))
        print("ALL PROBS:")
        for i in range(len(p)):
            print(f"  {id2label[i]}: {round(float(p[i]), 4)}")
        print("="*60)


# -----------------------------
# Feel free edit this line ↓↓↓
# -----------------------------

user_input = input("Enter a movie review sentence: ")

predict_texts(user_input)


# 10. Error Analysis

We examine misclassified examples to understand
model limitations.

Observed challenges:

- Comparative sentiment
- Sarcasm
- Pragmatic reversal
- Contrastive structures


In [ ]:
predict_texts([
    "I love it.",
    "I love it. Not.",
    "Great movie... said no one ever.",
    "This was amazing... until it wasn't.",
    "This movie is not as good as Titanic; it's already good enough.",
    "I thought it was good; it was actually terrible."
])